In [3]:
#Import Relevant Libraries
import math
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14

from qiskit import Aer
from qiskit import IBMQ
from qiskit import transpile

from qiskit.utils import QuantumInstance

from qiskit.opflow import StateFn
from qiskit.opflow import Gradient
from qiskit.opflow import CircuitSampler
from qiskit.opflow.primitive_ops import MatrixOp

from qiskit.circuit import Parameter
from qiskit.circuit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import RealAmplitudes

from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.optimizers import GradientDescent
from qiskit.algorithms.optimizers import QNSPSA

from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator

In [4]:
I = np.matrix([[1, 0], [0, 1]], dtype=complex)
X = np.matrix([[0, 1], [1, 0]], dtype=complex)
Y = np.matrix([[0, -1j], [1j, 0]], dtype=complex)
Z = np.matrix([[1, 0], [0, -1]], dtype=complex)

In [5]:
def Rz(theta):
    return np.matrix([[np.exp(1j*theta/2), 0], [0, np.exp(-1j*theta/2)]], dtype=complex)

In [6]:
print(Rz(np.pi))

[[6.123234e-17+1.j 0.000000e+00+0.j]
 [0.000000e+00+0.j 6.123234e-17-1.j]]


In [7]:
def createRho(N):
    size = int(np.power(2, N))
    lis = []
    count = 1
    for i in range(size):
        temp = []
        for j in range(size):
            temp.append(1)
            count = count + 1
        lis.append(temp)
    return np.matrix(lis)

In [8]:
def createKron(A, N):
    if (N==1):
        return A
    prod = A
    for i in range(1, N):
        prod = np.kron(prod, A)
    return prod

In [9]:
def createRhoBar(rho, thetalist, N):
    size = int(np.power(2, N))
    summ = np.zeros((size, size), dtype=complex)
    for theta in thetalist:
        summ += createKron(Rz(theta), N)*rho*createKron(Rz(-theta), N)
    summ = summ/len(thetalist)
    sumReal = np.zeros((size, size), dtype=int)
    for i in range(len(summ)):
        for j in range(len(summ)):
            sumReal[i, j] = np.real(summ[i, j])
            if (np.abs(sumReal[i, j]) <= 1e-1):
                sumReal[i, j] = 0
    return sumReal

In [10]:
def createPhiList(N):
    size = int(np.power(2, N))
    PhiList = []
    i = 1
    while i<2*size:
        PhiList.append(i*np.pi/size)
        i = i+2
    return PhiList

In [17]:
def createThetaList(N):
    size = int(np.power(2, N))
    ThetaList = []
    for i in range(size):
        ThetaList.append(i*np.pi/(size/2))
    return ThetaList

In [18]:
createRhoBar(createRho(1), createThetaList(1), 1)

array([[1, 0],
       [0, 1]])

In [19]:
createRhoBar(createRho(2), createThetaList(2), 2)

array([[1, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1]])

In [20]:
createRhoBar(createRho(3), createThetaList(3), 3)

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [184]:
createRhoBar(createRho(4), createPhiList(4), 4)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])